参考：https://github.com/XDTD/fund_crawler
http://fund.eastmoney.com/pingzhongdata/000001.js
公司列表(Github上直接打开好像会提示not found,复制到浏览器上方直接进入即可)：http://fund.eastmoney.com/js/jjjz_gs.js
基金列表：http://fund.eastmoney.com/js/fundcode_search.js
基金信息1：http://fund.eastmoney.com/pingzhongdata/'+code+'.js‘ 其中,code为6位整数，如000001的URL位=为http://fund.eastmoney.com/pingzhongdata/000001.js
基金信息2:http://fund.eastmoney.com/f10/tsdata_'+code+'.html'，同上
基金经理信息:http://fundf10.eastmoney.com/jjjl_'+code+'.html',同上


In [4]:
import requests
import re
import yaml
import logging
import json
logging.basicConfig(format='%(levelname)s : %(message)s',
                    level=logging.DEBUG,
                    handlers=[logging.StreamHandler()])


logger = logging.getLogger(__name__)


def get_resonse(url):
    """
    :param url: 网页URL
    :return: 爬取的文本信息
    """
    try:
        r = requests.get(url)
        r.raise_for_status()
        r.encoding = 'utf-8'
        return r.text
    except:
        print('Failed to get response to url!')
        return ''

def get_fund_info(code):
    failed_list = []
    data_list = {}
    url = 'http://fund.eastmoney.com/pingzhongdata/'+code+'.js'
    response = get_resonse(url)
    # 爬取失败等待再次爬取
    if response is '':
        return ''
    else:
        strs = re.findall(r'var(.*?);',response)
        for i in range(0,len(strs)):
            tmp = strs[i].split('=')
            var_name = tmp[0].strip()
            data_list[var_name] = [tmp[1]]
        return data_list

code = "151001"
data = get_fund_info(code)

manager_list = data.get('Data_currentFundManager',None)
if manager_list is None or len(manager_list)==0:
    print("找不到基金经理")
    exit()

for manager in manager_list:
    manager = json.loads(manager)[0]
    print("基金[%s]经理：#%r %s 管理时间%s 基金规模%s" %          
         (code,
          manager['id'],
          manager['name'],
          manager['workTime'],
          manager['fundSize']))          

DEBUG : Starting new HTTP connection (1): fund.eastmoney.com
DEBUG : http://fund.eastmoney.com:80 "GET /pingzhongdata/151001.js HTTP/1.1" 200 106424


基金[151001]经理：#'30413768' 袁曦 管理时间5年又255天 基金规模23.54亿(3只基金)


In [28]:
from bs4 import BeautifulSoup

def get_fund_manager(code):
    url = "https://fundf10.eastmoney.com/jjjl_{}.html".format(code)
    content = get_resonse(url)
    soup = BeautifulSoup(content)  
    results = soup.select(".detail .txt_cont .boxitem table tbody tr")
    # print(results[0])
    manager = results[1]
    tds = manager.find_all('td')
    for td in tds:
        print(td.text)

    
get_fund_manager("151001")



DEBUG : Starting new HTTPS connection (1): fundf10.eastmoney.com
DEBUG : https://fundf10.eastmoney.com:443 "GET /jjjl_151001.html HTTP/1.1" 200 41626


2021-03-05
2021-03-12
钱睿南 袁曦 
7天
-3.78%
